In [15]:
import pandas as pd
import numpy as np
import pdfplumber
import re
from collections import namedtuple

In [16]:
Line = (namedtuple('Line', 'Main_title  Keys Values'))

In [17]:
main = re.compile(r'(\d)\. (.*)')

In [18]:
under_title = re.compile(r'(\d.\d) (.*)')

In [19]:
key_value = re.compile(r'(.*):[\n|\s](.*)')

In [26]:
lines = []
with pdfplumber.open('Sledgehammer-Heat-Cure-Powder-SDS-US-20150522.pdf') as pdf:
    page = pdf.pages
    for page in pdf.pages:
        
        text = page.extract_text(x_tolerance=2, y_tolerance=1)
        #text=text.replace('· ', '').replace('  ',' ').split('\n')
        #print(text)
        for line in text.split('\n'):
            if main.search(line):
                num, Main_title = main.search(line).group(1), main.search(line).group(2)
        #if not key_value.search(line):
           # Under_title = line
            if key_value.search(line):
                Keys,Values = key_value.search(line).group(1),key_value.search(line).group(2)
            
                lines.append(Line(Main_title, Keys,Values))
        

FileNotFoundError: [Errno 2] No such file or directory: 'Sledgehammer-Heat-Cure-Powder-SDS-US-20150522.pdf'

In [22]:
print(text)

['Page 11/11', 'Safety Data Sheet (SDS)', 'OSHA HazCom Standard 29 CFR 1910.1200(g) and GHS Rev 03.', 'Issue date 02/09/2017 Reviewed on 02/09/2017', 'Trade name: 10N Sodium Hydroxide (NaOH 40%)', 'National regulations:', 'The product is subject to be classified according with the latest version of the regulations on', 'hazardous substances.', 'Chemical safety assessment: A Chemical Safety Assessment has not been carried out.', '16 Other Information', 'The information and recommendations in this safety data sheet are, to the best of our knowledge,', 'accurate as of the date of issue. Nothing herein shall be deemed to create warranty, expressed or', 'implied, and shall not establish a legally valid contractual relationship. It is the responsibility of the user', 'to determine applicability of this information and the suitability of the material or product for any', 'particular purpose.', 'Date of preparation / last revision: 02/09/2017 / 5', 'Abbreviations and acronyms:', 'ADR: The Euro

In [23]:
lines

[]

In [24]:
df = pd.DataFrame(lines)
df

""


In [25]:

for line in lines:
    print(line._asdict())

In [11]:
import json
with open('pdf_1_nested.json', 'w') as fp:
    json.dump(lines, fp)

In [16]:
def recur_dictify(frame):
    if len(frame.columns) == 1:
        if frame.values.size == 1: return frame.values[0][0]
        return frame.values.squeeze()
    grouped = frame.groupby(frame.columns[0])
    d = {k: recur_dictify(g.iloc[:,1:]) for k,g in grouped}
    return d
nested_dic = recur_dictify(df)
with open('pdf_1_nested.json', 'w') as fp:
    json.dump(nested_dic, fp)

TypeError: Object of type ndarray is not JSON serializable